In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd '/content/gdrive/My Drive/new results'

/content/gdrive/My Drive/new results


In [ ]:
!pip install numpy==1.23

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 73.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5


In [ ]:
!pip install -q -U "tensorflow-text==2.11.*"
!pip install -q tf-models-official==2.11.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 99.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 52.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-datasets 4.9.3 requires protobuf>=3.20, but you have protobuf 3.19.6 which is incompatible.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 3.

In [ ]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
#from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt
import sklearn

In [ ]:
from sklearn.model_selection import KFold
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
import re

In [ ]:
import pandas as pd
full = pd.read_csv("/content/gdrive/My Drive/sst2/rep/50K-1/twitter140.csv", encoding='latin-1',dtype = str)
text_list = full.iloc[:,5]
lab_list = full.iloc[:,0]
full.columns = ["label", "id", "date", "flag", "user", "text"]

can_df = pd.read_csv("ca.csv").dropna()

#can_df = balance(can_df)
ca_text_clean2 = can_df["text"].tolist()
ca_lab2 = can_df["labels"].tolist()
#ca_lab2 = [str(w) for w in ca_lab]

au_df = pd.read_csv("au.csv").dropna()
#can_df = balance(can_df)
au_text_clean2 = au_df["text"].tolist()
au_lab2 = au_df["labels"].tolist()
#au_lab2 = [str(w) for w in au_lab]

nz_df = pd.read_csv("nz.csv").dropna()
#can_df = balance(can_df)
nz_text_clean2 = nz_df["text"].tolist()
nz_lab2 = nz_df["labels"].tolist()
#nz_lab2 = [str(w) for w in nz_lab]


uk_df = pd.read_csv("uk.csv").dropna()
#can_df = balance(can_df)
uk_text_clean2 = uk_df["text"].tolist()
uk_lab2 = uk_df["labels"].tolist()
#uk_lab2 = [str(w) for w in uk_lab]

us_df = pd.read_csv("us.csv").dropna()
#can_df = balance(can_df)
us_text_clean2 = us_df["text"].tolist()
us_lab2 = us_df["labels"].tolist()
#us_lab2 = [str(w) for w in us_lab]

za_df = pd.read_csv("za.csv").dropna()
#can_df = balance(can_df)
za_text_clean2 = za_df["text"].tolist()
za_lab2 = za_df["labels"].tolist()
#za_lab2 = [str(w) for w in za_lab]


In [ ]:
len(can_df)

1834

In [ ]:
len(au_df)

2163

In [ ]:
len(nz_df)

329

In [ ]:
len(uk_df)

6442

In [ ]:
len(us_df)

17566

In [ ]:
len(za_df)

250

In [ ]:
print(ca_lab2[0:10])

[4, 4, 0, 0, 4, 4, 4, 4, 0, 0]


In [ ]:
###creating mixed dataset
df_50k_2 = pd.concat([can_df, au_df, nz_df, uk_df, us_df, za_df], ignore_index=True, sort=False)
samp_50k_2 = [*ca_text_clean2,*au_text_clean2,*nz_text_clean2,*uk_text_clean2,*us_text_clean2, *za_text_clean2]
samp_50k_lab_2 = [*ca_lab2,*au_lab2,*nz_lab2,*uk_lab2,*us_lab2, *za_lab2]

In [ ]:
from sklearn.metrics import log_loss
def pred_from_ftb(text, model):
  tf_batch = tf.convert_to_tensor(text, dtype = None, name = "sentence")
  tf_prob = model(tf_batch)
  tf_outputs = tf.nn.sigmoid(tf_prob)
  tf_predictions =tf.round(tf_outputs)
  pred_labs = [0,4]
  #preds = tf.argmax(tf_predictions, axis=1)
  preds = tf.transpose(tf_predictions).numpy().tolist()[0]
  lab_preds = [pred_labs[int(i)] for i in preds]
  prob_preds = tf.transpose(tf_outputs).numpy().tolist()[0]
  return lab_preds, prob_preds
  #print([i for i in preds])

def lab_check(preds, labs):
    # Check if both the lists have the same length
    if len(preds) != len(labs):
        print("length different:" ,len(preds), len(labs))


    # Check if all the elements of both the lists are same
    count = 0
    results = []
    for i in range(len(preds)):
        #for j in range(len(labs)):
        if preds[i] == labs[i]:
          results.append(True)
        else:
          results.append(False)
    # check if count is equal to length of list

    return results

def cross_entropy(prob_preds, labs):
  prob_tr = []
  for i in labs:
    if i == "0":
      prob_tr.append(0.0)
    elif i == "4":
      prob_tr.append(1.0)
    else:
      raise
  ce = log_loss(prob_tr, prob_preds)
  return ce


def valid_from_ftb(text, model):
  tf_batch = tf.convert_to_tensor(text, dtype = None, name = "sentence")
  tf_outputs = tf.nn.sigmoid(model(tf_batch))
  tf_predictions =tf.round(tf_outputs)
  pred_labs = [0,1]
  #preds = tf.argmax(tf_predictions, axis=1)
  preds = tf.transpose(tf_predictions).numpy().tolist()[0]
  return [pred_labs[int(i)] for i in preds]
  #print([i for i in preds])

In [ ]:
def outputs(data, fn,c):
    df = pd.DataFrame({"baseline": data[0],
                        "5-mixed": data[1],
                        "CA": data[2],
                        "AU": data[3],
                        "NZ": data[4],
                        "UK": data[5],
                        "US": data[6],
                        "ZA":data[7]})
    #print(fn + ":", df.mean())
    path = "za-new3-1_"+str(c)+"-"+fn + ".csv"
    df.to_csv(path)


In [ ]:
from sklearn.utils import resample
ind_list = list(range(len(text_list)))
import random
#random.Random(1234).shuffle(ind_list)
random.Random(333).shuffle(ind_list) #random sample from full data

In [ ]:
opt_epochs = [[3,4,6,7,5],[5,6,3,4,6],[2,2,2,2,2],[5,6,5,7,3],[3,6,3,3,4]]

In [ ]:
rand_seeds = [123,456,789,246,468,680,135,357,579,1024,101,102,103,104,105,106,107,108,109,2048,201,202,203,204,205,206,207,208,209,3096]

In [ ]:
#rand_seeds = [206,207,208,209,3096]

### precision and recall scores

In [ ]:
count = 0

for n in range(30):
  #n=a+24
  print(n)
  rands_text_2, rands_lab_2 = resample(samp_50k_2, samp_50k_lab_2, random_state = rand_seeds[n], n_samples = 200, replace = False)
  uk_text_clean, uk_lab = resample(uk_text_clean2, uk_lab2, random_state = rand_seeds[n], n_samples = 200 ,replace = False)
  us_text_clean, us_lab = resample(us_text_clean2, us_lab2, random_state = rand_seeds[n], n_samples = 200, replace = False)
  ca_text_clean, ca_lab = resample(ca_text_clean2, ca_lab2, random_state = rand_seeds[n], n_samples = 200, replace = False)
  au_text_clean, au_lab = resample(au_text_clean2, au_lab2, random_state = rand_seeds[n], n_samples = 200, replace = False)
  nz_text_clean, nz_lab = resample(nz_text_clean2, nz_lab2, random_state = rand_seeds[n], n_samples = 200, replace = False)
  za_text_clean, za_lab = resample(za_text_clean2, za_lab2, random_state = rand_seeds[n], n_samples = 200, replace = False)
  #print(nz_text_clean[0:5])

  CA_p0 = [] ##precision scores for negative label prediction
  AU_p0 = []
  NZ_p0 = []
  UK_p0 = []
  US_p0 = []
  Base_p0 = []
  Rand_p0 = []
  ZA_p0 = []

  CA_p1 = [] ##precision scores for positive label prediction
  AU_p1 = []
  NZ_p1 = []
  UK_p1 = []
  US_p1 = []
  Base_p1 = []
  Rand_p1 = []
  ZA_p1 = []

  CA_r0 = [] ##recall scores for negative label prediction
  AU_r0 = []
  NZ_r0 = []
  UK_r0 = []
  US_r0 = []
  Base_r0 = []
  Rand_r0 = []
  ZA_r0 = []

  CA_r1 = [] ##recall scores for positive label prediction
  AU_r1 = []
  NZ_r1 = []
  UK_r1 = []
  US_r1 = []
  Base_r1 = []
  Rand_r1 = []
  ZA_r1 = []

  for i in range(5):
    print(i)
    seeds = 111*(i+1)
    random.seed(seeds)
    ind_inds = random.sample(range(len(text_list)),16000)
    samp_inds = [ind_list[x] for x in ind_inds]
    samp_text = [text_list[i] for i in samp_inds]
    samp_text_clean = [' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",t).split()) for t in samp_text]
    samp_labs = [lab_list[j] for j in samp_inds]
    kf = KFold(n_splits=5)
    kf.get_n_splits(samp_inds)
    for j, (train_index, test_index) in enumerate(kf.split(samp_inds)):
  #if j == 1 :
      print(f"Fold {j+1}:")
      path_pref = '/content/gdrive/My Drive/new-train/full-r' + str(i+1)+"-fold"+ str(j+1) ##getting test data from the train-test split
      model_path = path_pref + "-bert-ft-twitter-50k-" + "-epoch" + str(opt_epochs[i][j])
      #print(model_path)
      test_index = resample(test_index, random_state = 123, n_samples = 200, replace = False)
      test_text_clean = np.array(samp_text_clean)[test_index]
      test_lab = np.array(samp_labs)[test_index]
      test_lab = test_lab.astype(int)

    #convert_to_1(train_lab)
      model = tf.saved_model.load(model_path)
      print("done")
      print("model predicting")
      baseline_preds, base_prob = pred_from_ftb(test_text_clean, model)
      print(type(baseline_preds))
      print(baseline_preds[0:10])
      print(type(test_lab))
      print(test_lab[0:10])
      ca_preds_tw, ca_prob = pred_from_ftb(ca_text_clean, model)
      au_preds_tw, au_prob= pred_from_ftb(au_text_clean, model)
      nz_preds_tw, nz_prob= pred_from_ftb(nz_text_clean, model)
      uk_preds_tw, uk_prob= pred_from_ftb(uk_text_clean, model)
      us_preds_tw, us_prob= pred_from_ftb(us_text_clean, model)
      samp_preds_rand, samp_prob_rand = pred_from_ftb(rands_text_2, model)
      za_preds_tw, za_prob= pred_from_ftb(za_text_clean, model)

      ca_f1_results = precision_recall_fscore_support(ca_lab, ca_preds_tw, labels=[0, 4])
      au_f1_results = precision_recall_fscore_support(au_lab, au_preds_tw, labels=[0, 4])
      nz_f1_results = precision_recall_fscore_support(nz_lab, nz_preds_tw, labels=[0, 4])
      uk_f1_results = precision_recall_fscore_support(uk_lab, uk_preds_tw, labels=[0, 4] )
      us_f1_results = precision_recall_fscore_support(us_lab, us_preds_tw, labels=[0, 4])
      samp_rand_results = precision_recall_fscore_support(rands_lab_2, samp_preds_rand, labels=[0, 4])
      #samp_strat_results =precision_recall_fscore_support(strat_lab_2, samp_preds_strat)
      baseline_results = precision_recall_fscore_support(test_lab, baseline_preds, labels=[0, 4])
      za_f1_results = precision_recall_fscore_support(za_lab, za_preds_tw, labels=[0, 4])

      CA_p0.append(ca_f1_results[0][0])
      AU_p0.append(au_f1_results[0][0])
      NZ_p0.append(nz_f1_results[0][0])
      UK_p0.append(uk_f1_results[0][0])
      US_p0.append(us_f1_results[0][0])
      Base_p0.append(baseline_results[0][0])
      Rand_p0.append(samp_rand_results[0][0])
      ZA_p0.append(za_f1_results[0][0])

      CA_p1.append(ca_f1_results[0][1])
      AU_p1.append(au_f1_results[0][1])
      NZ_p1.append(nz_f1_results[0][1])
      UK_p1.append(uk_f1_results[0][1])
      US_p1.append(us_f1_results[0][1])
      Base_p1.append(baseline_results[0][1])
      Rand_p1.append(samp_rand_results[0][1])
      ZA_p1.append(za_f1_results[0][1])

      CA_r0.append(ca_f1_results[1][0])
      AU_r0.append(au_f1_results[1][0])
      NZ_r0.append(nz_f1_results[1][0])
      UK_r0.append(uk_f1_results[1][0])
      US_r0.append(us_f1_results[1][0])
      Base_r0.append(baseline_results[1][0])
      Rand_r0.append(samp_rand_results[1][0])
      ZA_r0.append(za_f1_results[0][1])

      CA_r1.append(ca_f1_results[1][1])
      AU_r1.append(au_f1_results[1][1])
      NZ_r1.append(nz_f1_results[1][1])
      UK_r1.append(uk_f1_results[1][1])
      US_r1.append(us_f1_results[1][1])
      Base_r1.append(baseline_results[1][1])
      Rand_r1.append(samp_rand_results[1][1])
      ZA_r1.append(za_f1_results[0][1])


  count = count + 1
  print("count:", count)
  #print(len(CA_prc))
  #outputs([Base_prc, Rand_prc, CA_prc, AU_prc, NZ_prc, UK_prc, US_prc, ZA_prc],"auprc", count)
  outputs([Base_p0, Rand_p0, CA_p0, AU_p0, NZ_p0, UK_p0, US_p0, ZA_p0],"precision_0", count)
  outputs([Base_p1, Rand_p1, CA_p1, AU_p1, NZ_p1, UK_p1, US_p1, ZA_p1],"precision_1", count)
  outputs([Base_r0, Rand_r0, CA_r0, AU_r0, NZ_r0, UK_r0, US_r0, ZA_r0],"recall_0", count)
  outputs([Base_r1, Rand_r1, CA_r1, AU_r1, NZ_r1, UK_r1, US_r1, ZA_r1],"recall_1", count)

Streaming output truncated to the last 5000 lines.
<class 'numpy.ndarray'>
[0 0 0 0 4 0 4 4 0 4]
Fold 3:
done
model predicting
<class 'list'>
[0, 0, 4, 0, 4, 4, 0, 4, 0, 0]
<class 'numpy.ndarray'>
[0 0 4 0 4 4 0 4 0 0]
Fold 4:
done
model predicting
<class 'list'>
[4, 4, 0, 0, 0, 0, 0, 0, 4, 0]
<class 'numpy.ndarray'>
[4 4 0 0 0 0 0 0 4 0]
Fold 5:
done
model predicting
<class 'list'>
[4, 0, 0, 0, 4, 4, 0, 0, 4, 4]
<class 'numpy.ndarray'>
[4 4 0 0 4 4 0 0 4 4]
1
Fold 1:
done
model predicting
<class 'list'>
[4, 0, 0, 0, 0, 4, 0, 0, 0, 4]
<class 'numpy.ndarray'>
[4 4 0 4 0 4 4 4 0 0]
Fold 2:
done
model predicting
<class 'list'>
[0, 0, 4, 4, 0, 0, 0, 0, 4, 4]
<class 'numpy.ndarray'>
[0 0 4 4 0 0 0 0 4 4]
Fold 3:
done
model predicting
<class 'list'>
[0, 0, 4, 0, 4, 4, 0, 4, 0, 0]
<class 'numpy.ndarray'>
[0 0 4 0 4 4 0 4 0 0]
Fold 4:
done
model predicting
<class 'list'>
[4, 4, 0, 0, 0, 0, 0, 4, 0, 4]
<class 'numpy.ndarray'>
[4 4 0 4 0 0 0 4 0 4]
Fold 5:
done
model predicting
<class 'list'>
[0

### AUC scores

In [ ]:
from sklearn.metrics import roc_auc_score
count = 24
CA_auc = []
AU_auc = []
NZ_auc = []
UK_auc = []
US_auc = []
Base_auc = []
Rand_auc = []
ZA_auc = []
for n in range(30):
  #n=a+24
  #print(n)
  rands_text_2, rands_lab_2 = resample(samp_50k_2, samp_50k_lab_2, random_state = rand_seeds[n], n_samples = 200, replace = False)
  uk_text_clean, uk_lab = resample(uk_text_clean2, uk_lab2, random_state = rand_seeds[n], n_samples = 200 ,replace = False)
  us_text_clean, us_lab = resample(us_text_clean2, us_lab2, random_state = rand_seeds[n], n_samples = 200, replace = False)
  ca_text_clean, ca_lab = resample(ca_text_clean2, ca_lab2, random_state = rand_seeds[n], n_samples = 200, replace = False)
  au_text_clean, au_lab = resample(au_text_clean2, au_lab2, random_state = rand_seeds[n], n_samples = 200, replace = False)
  nz_text_clean, nz_lab = resample(nz_text_clean2, nz_lab2, random_state = rand_seeds[n], n_samples = 200, replace = False)
  za_text_clean, za_lab = resample(za_text_clean2, za_lab2, random_state = rand_seeds[n], n_samples = 200, replace = False)
  #print(nz_text_clean[0:5])

  CA_auc = []
  AU_auc = []
  NZ_auc = []
  UK_auc = []
  US_auc = []
  Base_auc = []
  Rand_auc = []
  ZA_auc = []

  for i in range(5):
    print(i)
    seeds = 111*(i+1)
    random.seed(seeds)
    ind_inds = random.sample(range(len(text_list)),16000)
    samp_inds = [ind_list[x] for x in ind_inds]
    samp_text = [text_list[i] for i in samp_inds]
    samp_text_clean = [' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",t).split()) for t in samp_text]
    samp_labs = [lab_list[j] for j in samp_inds]
    kf = KFold(n_splits=5)
    kf.get_n_splits(samp_inds)
    for j, (train_index, test_index) in enumerate(kf.split(samp_inds)):
  #if j == 1 :
      print(f"Fold {j+1}:")
      path_pref = '/content/gdrive/My Drive/new-train/full-r' + str(i+1)+"-fold"+ str(j+1)
      model_path = path_pref + "-bert-ft-twitter-50k-" + "-epoch" + str(opt_epochs[i][j])
      #print(model_path)
      test_index = resample(test_index, random_state = 123, n_samples = 200, replace = False)
      test_text_clean = np.array(samp_text_clean)[test_index]
      test_lab = np.array(samp_labs)[test_index]
      test_lab = test_lab.astype(int)

    #convert_to_1(train_lab)
      model = tf.saved_model.load(model_path)
     # print("done")
      #print("model predicting")
      baseline_preds, base_prob = pred_from_ftb(test_text_clean, model)
      #print(type(baseline_preds))
      #print(baseline_preds[0:10])
      #print(type(test_lab))
      #print(test_lab[0:10])
      ca_preds_tw, ca_prob = pred_from_ftb(ca_text_clean, model)
      au_preds_tw, au_prob= pred_from_ftb(au_text_clean, model)
      nz_preds_tw, nz_prob= pred_from_ftb(nz_text_clean, model)
      uk_preds_tw, uk_prob= pred_from_ftb(uk_text_clean, model)
      us_preds_tw, us_prob= pred_from_ftb(us_text_clean, model)
      samp_preds_rand, samp_prob_rand = pred_from_ftb(rands_text_2, model)
      za_preds_tw, za_prob= pred_from_ftb(za_text_clean, model)



      ca_auc_results = roc_auc_score(ca_lab, ca_prob)
      #print("example_socre:", ca_auc_results)
      au_auc_results = roc_auc_score(au_lab, au_prob)
      nz_auc_results = roc_auc_score(nz_lab, nz_prob)
      uk_auc_results = roc_auc_score(uk_lab, uk_prob)
      us_auc_results = roc_auc_score(us_lab, us_prob)
      samp_rand_results_auc = roc_auc_score(rands_lab_2, samp_prob_rand)
      #samp_strat_results_ac =accuracy_score(strat_lab_2, samp_preds_strat)
      baseline_results_auc = roc_auc_score(test_lab, base_prob)
      za_auc_results = roc_auc_score(za_lab, za_prob)

      #print("summarizing")
      CA_auc.append(ca_auc_results)
      AU_auc.append(au_auc_results)
      NZ_auc.append(nz_auc_results)
      UK_auc.append(uk_auc_results)
      US_auc.append(us_auc_results)
      Base_auc.append(baseline_results_auc)
      Rand_auc.append(samp_rand_results_auc)
      ZA_auc.append(za_auc_results)

  count = count + 1
  #print("count:", count)
  #print(len(CA_auc))
  outputs([Base_auc, Rand_auc, CA_auc, AU_auc, NZ_auc, UK_auc, US_auc, ZA_auc],"auc", count)

##Accuracy, Precision scores and recall scores

In [ ]:
for n in range(30):
  rands_text_2, rands_lab_2 = resample(samp_50k_2, samp_50k_lab_2, random_state = rand_seeds[n], n_samples = 200, replace = False)
  uk_text_clean, uk_lab = resample(uk_text_clean2, uk_lab2, random_state = rand_seeds[n], n_samples = 200 ,replace = False)
  us_text_clean, us_lab = resample(us_text_clean2, us_lab2, random_state = rand_seeds[n], n_samples = 200, replace = False)
  ca_text_clean, ca_lab = resample(ca_text_clean2, ca_lab2, random_state = rand_seeds[n], n_samples = 200, replace = False)
  au_text_clean, au_lab = resample(au_text_clean2, au_lab2, random_state = rand_seeds[n], n_samples = 200, replace = False)
  nz_text_clean, nz_lab = resample(nz_text_clean2, nz_lab2, random_state = rand_seeds[n], n_samples = 200, replace = False)
  za_text_clean, za_lab = resample(za_text_clean2, za_lab2, random_state = rand_seeds[n], n_samples = 200, replace = False)

  CA_acc = [] ##accuracy results
  AU_acc = []
  NZ_acc = []
  UK_acc = []
  US_acc = []
  Base_acc = []
  Rand_acc = []
  ZA_acc = []

  CA_ce = [] ##cross-entropy results
  AU_ce = []
  NZ_ce = []
  UK_ce = []
  US_ce = []
  Base_ce = []
  Rand_ce = []
  ZA_ce = []

  CA_p0 = [] ##precision score results for negative label prediciton
  AU_p0 = []
  NZ_p0 = []
  UK_p0 = []
  US_p0 = []
  Base_p0 = []
  Rand_p0 = []
  ZA_p0 = []

  CA_p1 = [] ##precision score results for positive label prediction
  AU_p1 = []
  NZ_p1 = []
  UK_p1 = []
  US_p1 = []
  Base_p1 = []
  Rand_p1 = []
  ZA_p1 = []

  CA_r0 = [] ##recall score results for negative label prediction
  AU_r0 = []
  NZ_r0 = []
  UK_r0 = []
  US_r0 = []
  Base_r0 = []
  Rand_r0 = []
  ZA_r0 = []

  CA_r1 = [] ##recall score results for positive label prediction
  AU_r1 = []
  NZ_r1 = []
  UK_r1 = []
  US_r1 = []
  Base_r1 = []
  Rand_r1 = []
  ZA_r1 = []

  CA_F0 = [] ##f1-score results for negative label prediciton
  AU_F0 = []
  NZ_F0 = []
  UK_F0 = []
  US_F0 = []
  Base_F0 = []
  Rand_F0 = []
  ZA_F0 = []

  CA_F1 = [] ##f1-score results for positive label prediciton
  AU_F1 = []
  NZ_F1 = []
  UK_F1 = []
  US_F1 = []
  Base_F1 = []
  Rand_F1 = []
  ZA_F1 = []

  for i in range(5):
    seeds = 111*(i+1)
    random.seed(seeds)
    ind_inds = random.sample(range(len(text_list)),16000)
    samp_inds = [ind_list[x] for x in ind_inds]
    samp_text = [text_list[i] for i in samp_inds]
    samp_text_clean = [' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",t).split()) for t in samp_text]
    samp_labs = [lab_list[j] for j in samp_inds]
    kf = KFold(n_splits=5)
    kf.get_n_splits(samp_inds)
    for j, (train_index, test_index) in enumerate(kf.split(samp_inds)): ##get test set (baseline test dataset)
  #if j == 1 :
      #print(f"Fold {j+1}:")
      path_pref = '/content/gdrive/My Drive/new-train/full-r' + str(i+1)+"-fold"+ str(j+1)
      model_path = path_pref + "-bert-ft-twitter-50k-" + "-epoch" + str(opt_epochs[i][j])
      test_index = resample(test_index, random_state = 123, n_samples = 200, replace = False)
      test_text_clean = np.array(samp_text_clean)[test_index]
      test_lab = np.array(samp_labs)[test_index]

    #convert_to_1(train_lab)
      model = tf.saved_model.load(model_path)
      #print("done")
      #print("model predicting")
      baseline_preds, base_prob = pred_from_ftb(test_text_clean, model)
      #print(type(baseline_preds))
      #print(baseline_preds[0:10])
      #print(type(test_lab))
      #print(test_lab[0:10])
      ca_preds_tw, ca_prob = pred_from_ftb(ca_text_clean, model)
      au_preds_tw, au_prob= pred_from_ftb(au_text_clean, model)
      nz_preds_tw, nz_prob= pred_from_ftb(nz_text_clean, model)
      uk_preds_tw, uk_prob= pred_from_ftb(uk_text_clean, model)
      us_preds_tw, us_prob= pred_from_ftb(us_text_clean, model)
      samp_preds_rand, samp_prob_rand = pred_from_ftb(rands_text_2, model)
      za_preds_tw, za_prob= pred_from_ftb(za_text_clean, model)
    #samp_preds_strat, samp_prob_strat = pred_from_ftb(strat_text_2, model)


  #pred_df["round " + str(i+1)]= [ca_preds_tw, au_preds_tw, nz_preds_tw, uk_preds_tw, us_preds_tw, samp_preds_rand]
      #print("done")
      #print("evaluate predictions...")
      ca_f1_results = precision_recall_fscore_support(ca_lab, ca_preds_tw, labels=['0', '4'])
      au_f1_results = precision_recall_fscore_support(au_lab, au_preds_tw, labels=['0', '4'])
      nz_f1_results = precision_recall_fscore_support(nz_lab, nz_preds_tw, labels=['0', '4'])
      uk_f1_results = precision_recall_fscore_support(uk_lab, uk_preds_tw, labels=['0', '4'] )
      us_f1_results = precision_recall_fscore_support(us_lab, us_preds_tw, labels=['0', '4'])
      samp_rand_results = precision_recall_fscore_support(rands_lab_2, samp_preds_rand, labels=['0', '4'])
    #samp_strat_results =precision_recall_fscore_support(strat_lab_2, samp_preds_strat)
      baseline_results = precision_recall_fscore_support(test_lab, baseline_preds, labels=['0', '4'])
      za_f1_results = precision_recall_fscore_support(za_lab, za_preds_tw, labels=['0', '4'])

  #eva_df= pd.DataFrame(data = zip(ca_tw_results, au_tw_results, nz_tw_results, uk_tw_results, us_tw_results, samp_rand_results, samp_strat_results, baseline_results),
  #                     columns = ['ca', 'au', 'nz', 'uk', 'us', 'random', 'stratified', 'baseline'])
      ca_ac_results = accuracy_score(ca_lab, ca_preds_tw)
      au_ac_results = accuracy_score(au_lab, au_preds_tw)
      nz_ac_results = accuracy_score(nz_lab, nz_preds_tw)
      uk_ac_results = accuracy_score(uk_lab, uk_preds_tw )
      us_ac_results = accuracy_score(us_lab, us_preds_tw)
      samp_rand_results_ac = accuracy_score(rands_lab_2, samp_preds_rand)
      #samp_strat_results_ac =accuracy_score(strat_lab_2, samp_preds_strat)
      baseline_results_ac = accuracy_score(test_lab, baseline_preds)
      za_ac_results = accuracy_score(za_lab, za_preds_tw)

      #print("done")
      #print("calculating cross-entropy loss")
      ca_ce = 1-cross_entropy(ca_prob, ca_lab)
      au_ce = 1-cross_entropy(au_prob, au_lab)
      nz_ce = 1-cross_entropy(nz_prob, nz_lab)
      uk_ce = 1-cross_entropy(uk_prob, uk_lab)
      us_ce = 1-cross_entropy(us_prob, us_lab)
      rand_ce = 1-cross_entropy(samp_prob_rand, rands_lab_2)
      #strat_ce = 1-cross_entropy(samp_prob_strat, strat_lab_2)
      base_ce = 1-cross_entropy(base_prob, test_lab)
      za_ce = 1-cross_entropy(za_prob, za_lab)

      #print("summarizing")
      CA_acc.append(ca_ac_results)
      AU_acc.append(au_ac_results)
      NZ_acc.append(nz_ac_results)
      UK_acc.append(uk_ac_results)
      US_acc.append(us_ac_results)
      Base_acc.append(baseline_results_ac)
      Rand_acc.append(samp_rand_results_ac)
      ZA_acc.append(za_ac_results)

      CA_ce.append(ca_ce)
      AU_ce.append(au_ce)
      NZ_ce.append(nz_ce)
      UK_ce.append(uk_ce)
      US_ce.append(us_ce)
      Base_ce.append(base_ce)
      Rand_ce.append(rand_ce)
      ZA_ce.append(za_ce)

      CA_p0.append(ca_f1_results[0][0])
      AU_p0.append(au_f1_results[0][0])
      NZ_p0.append(nz_f1_results[0][0])
      UK_p0.append(uk_f1_results[0][0])
      US_p0.append(us_f1_results[0][0])
      Base_p0.append(baseline_results[0][0])
      Rand_p0.append(samp_rand_results[0][0])
      ZA_p0.append(za_f1_results[0][0])

      CA_p1.append(ca_f1_results[0][1])
      AU_p1.append(au_f1_results[0][1])
      NZ_p1.append(nz_f1_results[0][1])
      UK_p1.append(uk_f1_results[0][1])
      US_p1.append(us_f1_results[0][1])
      Base_p1.append(baseline_results[0][1])
      Rand_p1.append(samp_rand_results[0][1])
      ZA_p1.append(za_f1_results[0][1])

      CA_r0.append(ca_f1_results[1][0])
      AU_r0.append(au_f1_results[1][0])
      NZ_r0.append(nz_f1_results[1][0])
      UK_r0.append(uk_f1_results[1][0])
      US_r0.append(us_f1_results[1][0])
      Base_r0.append(baseline_results[1][0])
      Rand_r0.append(samp_rand_results[1][0])
      ZA_r0.append(za_f1_results[1][0])

      CA_r1.append(ca_f1_results[1][1])
      AU_r1.append(au_f1_results[1][1])
      NZ_r1.append(nz_f1_results[1][1])
      UK_r1.append(uk_f1_results[1][1])
      US_r1.append(us_f1_results[1][1])
      Base_r1.append(baseline_results[1][1])
      Rand_r1.append(samp_rand_results[1][1])
      ZA_r1.append(za_f1_results[1][1])

      CA_F0.append(ca_f1_results[2][0])
      AU_F0.append(au_f1_results[2][0])
      NZ_F0.append(nz_f1_results[2][0])
      UK_F0.append(uk_f1_results[2][0])
      US_F0.append(us_f1_results[2][0])
      Base_F0.append(baseline_results[2][0])
      Rand_F0.append(samp_rand_results[2][0])
      ZA_F0.append(za_f1_results[2][0])

      CA_F1.append(ca_f1_results[2][1])
      AU_F1.append(au_f1_results[2][1])
      NZ_F1.append(nz_f1_results[2][1])
      UK_F1.append(uk_f1_results[2][1])
      US_F1.append(us_f1_results[2][1])
      Base_F1.append(baseline_results[2][1])
      Rand_F1.append(samp_rand_results[2][1])
      ZA_F1.append(za_f1_results[2][1])

  count = count + 1


  outputs([Base_acc, Rand_acc, CA_acc, AU_acc, NZ_acc, UK_acc, US_acc, ZA_acc],"accuracy", count)
  outputs([Base_ce, Rand_ce, CA_ce, AU_ce, NZ_ce, UK_ce, US_ce,ZA_ce],"CE",count)
  outputs([Base_p0, Rand_p0, CA_p0, AU_p0, NZ_p0, UK_p0, US_p0, ZA_p0],"precision_0",count)
  outputs([Base_p1, Rand_p1, CA_p1, AU_p1, NZ_p1, UK_p1, US_p1, ZA_p1],"precision_1", count)
  outputs([Base_r0, Rand_r0, CA_r0, AU_r0, NZ_r0, UK_r0, US_r0, ZA_r0],"recall_0",count)
  outputs([Base_r1, Rand_r1, CA_r1, AU_r1, NZ_r1, UK_r1, US_r1, ZA_r1],"recall_1",count)
  outputs([Base_F0, Rand_F0, CA_F0, AU_F0, NZ_F0, UK_F0, US_F0, ZA_F0],"f1_0",count)
  outputs([Base_F1, Rand_F1, CA_F1, AU_F1, NZ_F1, UK_F1, US_F1, ZA_F1],"f1_1",count)

## binary class count

In [ ]:
count = 0
CA_size = []
AU_size = []
NZ_size = []
UK_size = []
US_size = []
Base_size = []
Rand_size = []
ZA_size = []
for n in range(30):
  rands_text_2, rands_lab_2 = resample(samp_50k_2, samp_50k_lab_2, random_state = rand_seeds[n], n_samples = 200, replace = False)
  uk_text_clean, uk_lab = resample(uk_text_clean2, uk_lab2, random_state = rand_seeds[n], n_samples = 200 ,replace = False)
  us_text_clean, us_lab = resample(us_text_clean2, us_lab2, random_state = rand_seeds[n], n_samples = 200, replace = False)
  ca_text_clean, ca_lab = resample(ca_text_clean2, ca_lab2, random_state = rand_seeds[n], n_samples = 200, replace = False)
  au_text_clean, au_lab = resample(au_text_clean2, au_lab2, random_state = rand_seeds[n], n_samples = 200, replace = False)
  nz_text_clean, nz_lab = resample(nz_text_clean2, nz_lab2, random_state = rand_seeds[n], n_samples = 200, replace = False)
  za_text_clean, za_lab = resample(za_text_clean2, za_lab2, random_state = rand_seeds[n], n_samples = 200, replace = False)

  CA_size.append(ca_lab.count("0"))
  AU_size.append(au_lab.count("0"))
  NZ_size.append(nz_lab.count("0"))
  UK_size.append(uk_lab.count("0"))
  US_size.append(us_lab.count("0"))
  Rand_size.append(rands_lab_2.count("0"))
  ZA_size.append(za_lab.count("0"))




df = pd.DataFrame({"baseline": data[0],
                        "5-mixed": Rand_size,
                        "CA": CA_size,
                        "AU": AU_size,
                        "NZ": NZ_size,
                        "UK": UK_size,
                        "US": US_size,
                        "ZA":ZA_size})

df.to_csv("za-runs-class-record.csv")
print(df.describe())



          5-mixed          CA          AU         NZ          UK          US  \
count   30.000000   30.000000   30.000000  30.000000   30.000000   30.000000   
mean   101.166667  101.933333   94.866667  82.866667  105.466667  101.066667   
std      7.278136    5.860054    5.751961   4.703142    6.257318    6.362245   
min     87.000000   88.000000   79.000000  75.000000   93.000000   88.000000   
25%     96.250000   98.000000   93.000000  80.000000  100.500000   97.250000   
50%    101.000000  102.500000   95.000000  82.500000  105.500000  101.000000   
75%    107.000000  106.750000   97.750000  84.750000  110.000000  104.000000   
max    116.000000  111.000000  104.000000  96.000000  122.000000  117.000000   

              ZA  
count  30.000000  
mean   88.466667  
std     3.645813  
min    81.000000  
25%    86.000000  
50%    88.500000  
75%    91.000000  
max    99.000000  
